In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, k_means
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.feature_selection import SelectKBest
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./data/mlb-pitches.csv')
df.drop(columns = ['Unnamed: 0'], inplace = True) 

pd.set_option('max_columns', None)

print(df.shape)
df.head(3)
#list(df.columns.values)
#df.info()

(708653, 76)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,post_away_score,post_home_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bauer_units,is_strike,pitch_count,swing_miss,plate_-x,pfx_-x
0,FF,2021-10-03,92.3,1.40,6.80,"Smith, Will",596019,519293,out,hit_into_play,1.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,X,5.0,ground_ball,1,2,2021,8.28,16.56,-0.69,2.83,NaN,NaN,NaN,2,9,Top,77.54,155.31,-6.833043,-134.166485,-7.361843,9.708393,26.562803,-14.083224,3.32,1.51,13.0,95.2,-13.0,92.8,2330.0,6.5,632254,54.03,0.174,0.158,0.0,1.0,0.0,0.0,2.0,61,4,4-Seam Fastball,5,0,0,5,Infield shift,Strategic,148.0,0.0,-0.073,25.243770,1,1-2,0,0.69,-8.28
1,SL,2021-10-03,80.6,1.60,6.64,"Smith, Will",596019,519293,NaN,foul,4.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,1,2021,-9.24,5.76,-0.71,2.62,NaN,NaN,NaN,2,9,Top,NaN,NaN,-3.700232,-117.430885,-3.266842,-6.531123,19.793390,-27.369114,3.32,1.51,108.0,75.3,75.0,81.2,2254.0,6.4,632254,54.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,Slider,5,0,0,5,Infield shift,Strategic,315.0,0.0,-0.027,27.965261,1,1-1,0,0.71,9.24
2,CU,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,NaN,foul,5.0,"Francisco Lindor grounds out, third baseman Eh...",R,R,L,ATL,NYM,S,NaN,NaN,1,0,2021,-7.80,-6.12,-0.04,2.46,NaN,NaN,NaN,2,9,Top,NaN,NaN,-1.977183,-109.901781,-1.155694,-4.872924,20.602334,-36.262184,3.32,1.51,157.0,83.5,65.0,75.2,1940.0,6.2,632254,54.34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,Curveball,5,0,0,5,Infield shift,Standard,328.0,0.0,-0.020,25.695364,1,1-0,0,0.04,7.80


In [3]:
#df[df.columns[df.isnull().any()]].isnull().sum()
#df.dropna(inplace = True)
df.dropna(subset = ['release_extension', 'release_spin_rate', 'if_fielding_alignment'], 
          inplace = True)

# Fill nulls with none so they aren't dropped
df['bb_type'] = df['bb_type'].fillna('not_in_play').astype(str)
df['events'] = df['events'].fillna('none').astype(str)
df.fillna(value = 0, inplace = True)


df = pd.get_dummies(df, columns = ['p_throws', 'type', 'stand', 'events', 
                                   'description', 'bb_type', 'if_fielding_alignment', 
                                   'of_fielding_alignment'])

# Drop pitch types Fasball, knuckleball, eephus, and screwball
df['pitch_type'] = df['pitch_type'].map({'FS': 8, 'CH': 7, 'KC': 6, 'CU': 5,
                                         'SL': 4, 'FC': 3, 'SI': 2, 'FF': 1})

print(df.shape)
df.head()

(702437, 124)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,zone,des,game_type,home_team,away_team,hit_location,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,post_away_score,post_home_score,spin_axis,delta_home_win_exp,delta_run_exp,bauer_units,is_strike,pitch_count,swing_miss,plate_-x,pfx_-x,p_throws_L,p_throws_R,type_B,type_S,type_X,stand_L,stand_R,events_catcher_interf,events_caught_stealing_2b,events_caught_stealing_3b,events_caught_stealing_home,events_double,events_double_play,events_game_advisory,events_home_run,events_none,events_other_out,events_out,events_passed_ball,events_pickoff_1b,events_pickoff_2b,events_pickoff_3b,events_pickoff_caught_stealing_2b,events_pickoff_caught_stealing_3b,events_sac_bunt,events_sac_bunt_double_play,events_sac_fly_double_play,events_single,events_stolen_base_2b,events_strikeout,events_strikeout_double_play,events_triple,events_triple_play,events_walk,events_wild_pitch,description_ball,description_bunt_foul_tip,description_called_strike,description_foul,description_foul_pitchout,description_hit_by_pitch,description_hit_into_play,description_missed_bunt,description_pitchout,description_swinging_strike,bb_type_fly_ball,bb_type_ground_ball,bb_type_line_drive,bb_type_not_in_play,bb_type_popup,if_fielding_alignment_Infield shift,if_fielding_alignment_Standard,if_fielding_alignment_Strategic,of_fielding_alignment_4th outfielder,of_fielding_alignment_Standard,of_fielding_alignment_Strategic
0,1,2021-10-03,92.3,1.40,6.80,"Smith, Will",596019,519293,1.0,"Francisco Lindor grounds out, third baseman Eh...",R,ATL,NYM,5.0,1,2,2021,8.28,16.56,-0.69,2.83,0.0,0.0,0.0,2,9,Top,77.54,155.31,-6.833043,-134.166485,-7.361843,9.708393,26.562803,-14.083224,3.32,1.51,13.0,95.2,-13.0,92.8,2330.0,6.5,632254,54.03,0.174,0.158,0.0,1.0,0.0,0.0,2.0,61,4,4-Seam Fastball,5,0,0,5,148.0,0.000,-0.073,25.243770,1,1-2,0,0.69,-8.28,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1
1,4,2021-10-03,80.6,1.60,6.64,"Smith, Will",596019,519293,4.0,"Francisco Lindor grounds out, third baseman Eh...",R,ATL,NYM,0.0,1,1,2021,-9.24,5.76,-0.71,2.62,0.0,0.0,0.0,2,9,Top,0.00,0.00,-3.700232,-117.430885,-3.266842,-6.531123,19.793390,-27.369114,3.32,1.51,108.0,75.3,75.0,81.2,2254.0,6.4,632254,54.15,0.000,0.000,0.0,0.0,0.0,0.0,0.0,61,3,Slider,5,0,0,5,315.0,0.000,-0.027,27.965261,1,1-1,0,0.71,9.24,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1
2,5,2021-10-03,75.5,1.46,6.88,"Smith, Will",596019,519293,5.0,"Francisco Lindor grounds out, third baseman Eh...",R,ATL,NYM,0.0,1,0,2021,-7.80,-6.12,-0.04,2.46,0.0,0.0,0.0,2,9,Top,0.00,0.00,-1.977183,-109.901781,-1.155694,-4.872924,20.602334,-36.262184,3.32,1.51,157.0,83.5,65.0,75.2,1940.0,6.2,632254,54.34,0.000,0.000,0.0,0.0,0.0,0.0,0.0,61,2,Curveball,5,0,0,5,328.0,0.000,-0.020,25.695364,1,1-0,0,0.04,7.80,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0
3,5,2021-10-03,75.0,1.53,6.83,"Smith, Will",596019,519293,12.0,"Francisco Lindor grounds out, third baseman Eh...",R,ATL,NYM,0.0,0,0,2021,-8.28,-8.28,2.10,3.89,0.0,0.0,0.0,2,9,Top,0.00,0.00,2.375830,-109.205830,2.277617,-5.902656,19.427562,-38.284747,3.31,1.48,0.0,0.0,0.0,74.5,2017.0,5.9,632254,54.61,0.000,0.000,0.0,0.0,0.0,0.0,0.0,61,1,Curveball,5,0,0,5,330.0,0.000,0.016,26.893333,0,0-0,0,-2.10,8.28,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
4,1,2021-10-03,91.2,1.49,6.66,"S

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 702437 entries, 0 to 708651
Columns: 124 entries, pitch_type to of_fielding_alignment_Strategic
dtypes: float64(42), int64(17), object(9), uint8(56)
memory usage: 407.3+ MB


# Pitch Numbers

1. 4-seam fastball
2. Sinker
3. Cutter
4. Slider
5. Curveball
6. Knuckle Curve
7. Changeup
8. Split-Finger

### # of Pitches

In [5]:
df.pitch_type.value_counts()

1    248739
4    135610
2    108683
7     79978
5     58444
3     46575
6     13586
8     10822
Name: pitch_type, dtype: int64

### Velocity

In [6]:
df.groupby('pitch_type')['release_speed'].mean()

pitch_type
1    93.701701
2    93.002682
3    88.532404
4    84.661462
5    78.743339
6    81.071603
7    84.815458
8    85.706644
Name: release_speed, dtype: float64

### Spin Rate

In [7]:
df.groupby('pitch_type')['release_spin_rate'].mean()

pitch_type
1    2273.273680
2    2123.278866
3    2371.815674
4    2416.451714
5    2506.650862
6    2536.233770
7    1749.841731
8    1365.298743
Name: release_spin_rate, dtype: float64

### Spin Axis

In [8]:
df.groupby('pitch_type')['spin_axis'].mean()

pitch_type
1    192.876497
2    194.738506
3    174.087815
4    145.986594
5    124.633666
6     77.354924
7    195.125722
8    227.108575
Name: spin_axis, dtype: float64

### HB

In [9]:
df.groupby('pitch_type')['pfx_-x'].mean()

pitch_type
1    2.851918
2    5.645708
3   -1.369095
4   -3.217669
5   -3.871916
6   -6.285912
7    3.620740
8    9.603515
Name: pfx_-x, dtype: float64

### VB

In [10]:
df.groupby('pitch_type')['pfx_z'].mean()

pitch_type
1    16.101541
2     9.225791
3     7.945734
4     1.718492
5    -8.751586
6   -11.299573
7     6.824918
8     3.874884
Name: pfx_z, dtype: float64

### Pitch Data for Model

In [11]:
pitch_data = df[['pitch_type', 'release_speed', 'release_spin_rate', 'spin_axis', 
                 'pfx_-x', 'pfx_z','release_extension', 'release_pos_x', 'release_pos_z',
                 'p_throws_L', 'p_throws_R']]

pitch_data.dropna(inplace = True)

print(pitch_data.shape)
pitch_data.head()

(702437, 11)


,pitch_type,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,p_throws_L,p_throws_R
0,1,92.3,2330.0,148.0,-8.28,16.56,6.5,1.40,6.80,1,0
1,4,80.6,2254.0,315.0,9.24,5.76,6.4,1.60,6.64,1,0
2,5,75.5,1940.0,328.0,7.80,-6.12,6.2,1.46,6.88,1,0
3,5,75.0,2017.0,330.0,8.28,-8.28,5.9,1.53,6.83,1,0
4,1,91.2,2281.0,143.0,-7.56,15.36,6.3,1.49,6.66,1,0


## Model

In [12]:
X = pitch_data

ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_scaled[:,:] *= -1

pca = PCA().fit(X_scaled)
principal_components = pca.transform(X_scaled)

In [13]:
model = pd.DataFrame(data = principal_components, columns = X.columns)

print(model.shape)
model.head()

(702437, 11)


,pitch_type,release_speed,release_spin_rate,spin_axis,pfx_-x,pfx_z,release_extension,release_pos_x,release_pos_z,p_throws_L,p_throws_R
0,2.086906,-2.167346,-1.155668,0.546232,-1.113156,-0.752438,0.072414,0.177642,0.064072,-0.416268,2.220446e-16
1,1.923821,-0.927518,2.033847,1.074533,0.112719,-2.114986,0.562505,0.621797,0.184684,-0.187711,-2.220446e-16
2,2.454914,0.257007,3.016992,1.635591,-0.212986,-2.035787,-0.092934,1.182466,0.645277,-0.191664,-2.220446e-16
3,2.555803,0.470073,2.986381,2.111015,0.294334,-1.845630,-0.169199,1.167534,0.638886,-0.150438,-2.220446e-16
4,2.188637,-1.972124,-0.997722,0.694257,-0.815425,-0.385978,-0.004080,0.307133,-0.054374,-0.373888,2.220446e-16


In [14]:
km = KMeans(n_clusters = 4,
            random_state = 0)

km.fit(model)
label = km.fit_predict(model)

print('label:', label)
print('Number of iterations:', km.n_iter_)
print('Number of features:', km.n_features_in_)
print('Number of clusters:', km.n_clusters)
print('Inertia:', km.inertia_)
#print('Silhouette Score:', silhouette_score(X_scaled, labels, metric = 'euclidean'))

label: [3 3 3 ... 0 0 0]
Number of iterations: 12
Number of features: 11
Number of clusters: 4
Inertia: 3582384.5427419646


In [15]:
centroids = km.cluster_centers_
print("Predicted clusters to points: ", label[:10], '\n')
print('Location of centroids:', '\n')
print(centroids)

Predicted clusters to points:  [3 3 3 3 3 3 3 3 3 0] 

Location of centroids: 

[[-1.65254097e+00 -7.83359923e-01 -1.66571365e-01  7.23056640e-02
   5.93235313e-02 -1.88514605e-02 -2.27779292e-02  6.54585487e-02
   2.93099426e-02  4.34961521e-02  1.74342422e-17]
 [-1.07298921e+00  1.16259146e+00  2.50034607e+00 -3.74082528e-02
  -5.48036092e-01  3.99926894e-01  2.38370373e-01 -3.68600197e-01
  -9.09648890e-03 -1.11444652e-03  1.46304342e-18]
 [ 1.67824226e-01  2.43746863e+00 -1.14738877e+00 -4.67060367e-02
  -7.77008673e-02 -2.93030516e-02 -4.92750826e-02  5.04367153e-02
  -4.51171721e-02  3.10519209e-03  9.06247903e-17]
 [ 2.47860552e+00 -1.00343310e+00  2.44409484e-01 -5.43088250e-02
   1.45917519e-01 -7.96179185e-02 -9.39282878e-03 -8.74724385e-03
  -5.57744393e-03 -6.12672593e-02 -2.75343930e-19]]


### Cluster Labels

Label:

0. Fastballs (FF, SI, FC) 1.551 
1. Off Speed (CH, Splitter) 6.868
2. Breaking (SL, CU, **KC**) 4.371
3. FB's & BB's (FC, SL, CU) 3.093

In [16]:
X['label'] = label
model['label'] = label

#model.groupby(by = 'label').mean().T
X.groupby(by = 'label').mean().T

label,0,1,2,3
pitch_type,1.551675,6.868443,4.371561,3.093144
release_speed,93.443728,85.488628,82.890600,87.916874
release_spin_rate,2258.601901,1707.955093,2500.481984,2183.243271
spin_axis,212.357236,238.815633,77.125392,174.952073
pfx_-x,8.061048,12.699241,-7.882598,-5.133033
pfx_z,13.497639,5.342786,-2.666940,8.388967
release_extension,6.365056,6.310010,6.267418,6.253043
release_pos_x,-1.801469,-1.948153,-1.843837,1.983728
release_pos_z,5.867524,5.701126,5.830007,5.906349
p_throws_L,0.000000,0.001101,0.000124,1.000000
